# Lab 6

## Objective
In and after this lab session you will

1. train a Gaussian NBC with the EM algorithm
2. compare the results you get to those of the k-Means clustering provided in SciKitLearn
3. discuss the classifiers from this lab session and those from the previous session (supervised learning of NBCs) in a brief report

## Background and Tools
The EM (Expectation - Maximisation) algorithm solves the problem of not being able to compute the Maximum Likelihood Estimates for unknown classes directly by iterating over two steps until there is no significant change in step 2 observable:

1. Compute the expected outcome for each example / sample given estimates for priors and distribution (essentially, the likelihoods for observing the sample assuming an estimated distribution).  
2. Compute new estimates for your priors and distributions (in the case of a Gaussian NBC, new means and variances are needed) based on the estimated expected values for how much each sample belongs to the respective distribution.

You can find the algorithm stated explicitly as given in Murphy, "Machine Learning - A probabilistic perspective", pp 352 - 353 HERE (Link to come).

One special case of the EM algorithm is k-Means clustering, for which an implementation can be found in SciKitLearn.

## Your implementation task
1. Implement the EM-algorithm to find a Gaussian NBC for the digits dataset from SciKitLearn (you can of course also use the MNIST_Light set from Lab 5, but for initial tests the digits data set is more convenient, since it is smaller in various aspects). You may assume (conditional) independence between the attributes, i.e., the covariances can be assumed to be simply the variances over each attribute. Split the data set in 70% training and 30% test data.

### SciKitLearn Digits <a name='sklearn1'></a>
1. SciKitLearn digits: just load it and use it as is. Use 70% of the data for training and 30% for testing. Run all of the four classifiers and compare and discuss the results.

In [1]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

min_max_scaler = MinMaxScaler()
X, y = load_digits(n_class=10, return_X_y=True)
X= min_max_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=42)

### PCA on the data

In [4]:
import pandas as pd
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(X)
principal_df = pd.DataFrame(data = principalComponents, 
                            columns = ['PC1', 'PC2', 'PC3'])
target_df = pd.DataFrame(data=y, columns=['Target'])
final_df = pd.concat([principal_df, target_df], axis=1)

In [5]:
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "browser"
fig = px.scatter_3d(final_df, x='PC1', y='PC2', z='PC3', color='Target')
fig.show()

ModuleNotFoundError: No module named 'plotly'

### EM-algorithm

In [2]:
from scipy.stats import norm
import numpy as np
import sys


def EM_algorithm(X, k_classes, N=200,  eps=5e-2):
    
    def E_step(X, k_classes, mu, sig, prior):
        r = np.zeros([len(X), k_classes])
        for i, xi in enumerate(X):
            num = np.prod([norm.pdf(xi, mu[kcl], np.sqrt(sig[kcl])) for kcl in range(k_classes)], axis=1)
            num = [prior[kcl]*num[kcl] for kcl in range(k_classes)]
            den = np.sum(num)
            r[i, :] = num/den            
        return r

    def M_step(X, r):
        N, k_classes = r.shape
        prior = {kcl:np.sum(r[:, kcl])/N for kcl in range(k_classes)}
        mu = {}
        sig = {}
        
        for kcl in range(k_classes):
            mu[kcl] = np.sum([r[i, kcl]*xi for i, xi in enumerate(X)], axis=0)/np.sum(r[:, kcl])
            sig[kcl] = np.sum([r[i, kcl]*np.diag(np.outer(xi, xi)) for i, xi in enumerate(X)], axis= 0)/np.sum(r[:, kcl]) - np.diag(np.outer(mu[kcl], mu[kcl])) + eps
        return mu, sig, prior
    
    
    # Initial values
    mu = {kcl:np.random.uniform(1, size=X[0].size) for kcl in range(k_classes)}
    sig = {kcl:np.random.uniform(0.5, size=X[0].size) for kcl in range(k_classes)}
    prior = {kcl:1/k_classes for kcl in range(k_classes)} #Assuming equal distribution
    
    for iteration in range(N):
        r = E_step(X_train, k_classes, mu, sig, prior)
        mu, sig, prior = M_step(X_train, r)
        
    class GNB_classifier:
        def __init__(self, mu, sig, prior, k_classes, eps):
            self.mu = mu
            self.sig = sig
            self.prior = prior
            self.eps = eps
            self.classes = k_classes
        
        
        def predict(self, X):
            y_pred = []
            for x in X:
                prob = [self._posterior(x, kcl) for kcl in range(self.classes)]
                y_pred.append(np.argmax(prob))
            return np.asarray(y_pred)
    
    
        def _posterior(self, x, ci):
            return self.prior[ci]*np.prod([norm.pdf(x[i], self.mu[ci][i], self.sig[ci][i] + self.eps) for i in range(len(x))])
    
    return GNB_classifier(mu, sig, prior, k_classes, eps)

2. Use the result of the EM-algorithm (the found distribution parameters) to cluster the training data (essentially, using the resulting classifier to do a prediction over the training data). Produce a confusion matrix over the known labels for the training data and your EM-generated clusters. What do you see?

In [3]:
from sklearn import metrics

gnb_clf = EM_algorithm(X_train, k_classes=10, N=200)
gnb_pred = gnb_clf.predict(X_test)
measure = metrics.homogeneity_completeness_v_measure(y_test, gnb_pred)

print("Homogeneity Score: ", measure[0])
print("Completeness Score: ", measure[1])
print("V-measure: ", measure[2])

Homogeneity Score:  0.7268138846521567
Completeness Score:  0.7409117116451881
V-measure:  0.7337950917850213


3. If necessary, find a way to "repair" the cluster assignments so that you can do a prediction run over the test data, from which you can compare the results with your earlier implementation of the Gaussian NBC.

In [8]:
def repair(X_train, y_train, y_pred):
    k_classes = list(set(y_train))
    c_true = {}
    c_pred = {}
    for kcl in k_classes:
        c_true[kcl] = np.mean([X for i, X in enumerate(X_train) if y_train[i] == kcl], axis=0)
        c_pred[kcl] = np.mean([X for i, X in enumerate(X_train) if y_pred[i] == kcl], axis=0)

    pred2true = {}
    for kcl in k_classes:
        pred2true[kcl] = np.argmin([np.linalg.norm(c_pred[kcl] - c_true_v) for c_true_v in c_true.values()])
    
    y_new = []
    for y in y_pred:
        y_new.append(pred2true[y])
    return np.asarray(y_new)


def repair_ind(X_train, y_train, y_pred):
    k_classes = list(set(y_train))
    pred2true = {}
    for kcl in k_classes:
        indices = [i for i, x in enumerate(X_train) if y_pred[i] == kcl]
        unique, counts= np.unique(y_train[indices], return_counts=True)
        pred2true[kcl] = unique[np.argmax(counts)]
        
    y_new = []
    for y in y_pred:
        y_new.append(pred2true[y])
    return np.asarray(y_new)

In [13]:
em_pred = repair(X_test, y_test, gnb_pred)
em_pred2 = repair_ind(X_test, y_test, gnb_pred)

In [15]:
measure = metrics.homogeneity_completeness_v_measure(y_test, em_pred)

print("Homogeneity Score: ", measure[0])
print("Completeness Score: ", measure[1])
print("V-measure: ", measure[2])
print('\n')
print("Classification report:\n%s\n"
      % (metrics.classification_report(y_test, em_pred)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, em_pred))

Homogeneity Score:  0.7237721908319448
Completeness Score:  0.7752544056226649
V-measure:  0.748629251724719


Classification report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        53
           1       0.54      0.80      0.65        50
           2       0.80      0.87      0.84        47
           3       0.00      0.00      0.00        54
           4       0.95      0.90      0.92        60
           5       0.94      0.68      0.79        66
           6       0.98      0.98      0.98        53
           7       0.88      0.93      0.90        55
           8       0.61      0.51      0.56        43
           9       0.42      0.80      0.55        59

    accuracy                           0.74       540
   macro avg       0.71      0.74      0.72       540
weighted avg       0.72      0.74      0.72       540


Confusion matrix:
[[50  0  0  0  2  1  0  0  0  0]
 [ 0 40 10  0  0  0  0  0  0  0]
 [ 0  0 41  0  0  0  0

/Users/justinma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
measure = metrics.homogeneity_completeness_v_measure(y_test, em_pred2)

print("Homogeneity Score: ", measure[0])
print("Completeness Score: ", measure[1])
print("V-measure: ", measure[2])
print('\n')
print("Classification report:\n%s\n"
      % (metrics.classification_report(y_test, em_pred2)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, em_pred2))

Homogeneity Score:  0.7237721908319448
Completeness Score:  0.7752544056226649
V-measure:  0.748629251724719


Classification report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        53
           1       0.54      0.80      0.65        50
           2       0.80      0.87      0.84        47
           3       0.00      0.00      0.00        54
           4       0.95      0.90      0.92        60
           5       0.94      0.68      0.79        66
           6       0.98      0.98      0.98        53
           7       0.88      0.93      0.90        55
           8       0.61      0.51      0.56        43
           9       0.42      0.80      0.55        59

    accuracy                           0.74       540
   macro avg       0.71      0.74      0.72       540
weighted avg       0.72      0.74      0.72       540


Confusion matrix:
[[50  0  0  0  2  1  0  0  0  0]
 [ 0 40 10  0  0  0  0  0  0  0]
 [ 0  0 41  0  0  0  0

/Users/justinma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


4. Use now also the k-Means implementation from SciKitLearn and compare the results to yours (they should be similar at least in the sense that there are classes that are more clearly separated from the rest than others for both approaches). 

In [17]:
from sklearn.cluster import KMeans

km_clf = KMeans(n_clusters=10, random_state=42).fit(X_train)
km_pred = km_clf.predict(X_test)
km_pred = repair(X_test, y_test, km_pred)

measure = metrics.homogeneity_completeness_v_measure(y_test, km_pred)

print("Homogeneity Score: ", measure[0])
print("Completeness Score: ", measure[1])
print("V-measure: ", measure[2])
print('\n')
print("Classification report:\n%s\n"
      % (metrics.classification_report(y_test, km_pred)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, km_pred))

Homogeneity Score:  0.7561055338287734
Completeness Score:  0.8081139384915079
V-measure:  0.7812451278991431


Classification report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        53
           1       0.77      0.80      0.78        50
           2       0.79      0.87      0.83        47
           3       0.00      0.00      0.00        54
           4       0.96      0.90      0.93        60
           5       0.94      0.68      0.79        66
           6       0.96      0.98      0.97        53
           7       0.87      1.00      0.93        55
           8       0.82      0.86      0.84        43
           9       0.41      0.81      0.55        59

    accuracy                           0.79       540
   macro avg       0.75      0.79      0.76       540
weighted avg       0.75      0.79      0.76       540


Confusion matrix:
[[52  0  0  0  1  0  0  0  0  0]
 [ 0 40 10  0  0  0  0  0  0  0]
 [ 0  0 41  0  0  0  

/Users/justinma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
